In [ ]:
%run ../chap15/gan.ipynb
%run ../chap15/NLP_dataset.ipynb
%run ../chap15/NLP_Doc.ipynb

In [ ]:
#no_test
#RnnBasicModel('am_4',  ad, ['multi_layer_RNN', {'recur_size':4,  'outseq':False}])
class multi_layer_RNN(NLP_Doc):
    def __init__(self, name, dataset, hconfigs, JJ=0, show_maps=False,
                 l2_decay=0, l1_decay=0, dump_structure=True, word_vector_dimension=100, window_size=1, negative=5, load=0, sentence_vector_dimension=100):
        

        super(multi_layer_RNN, self).__init__(name, dataset, hconfigs, show_maps,
                                          l2_decay, l1_decay)
        
    
    
    def alloc_multi_layer_RNN_layer(self, input_shape, hconfig):
        inseq = get_conf_param(hconfig, 'inseq', True)
        outseq = get_conf_param(hconfig, 'outseq', True)
        use_state = get_conf_param(hconfig, 'use_state', False)
        
        if 'floor' in hconfig:
            self.floor=len(hconfig['floor']) # 딕셔너리로 층 마다 노드 개수 등 입력받기 [1, {recur_size}
        """
        a_={1:['1',{'re':10}]}
        if 1 in a_:
            print(a_[1][1]['re'])
        """
        if inseq:
            timesteps1, timefeats = input_shape
        else:
            timesteps1 = get_conf_param(hconfig, 'timesteps') + 1
            timefeats = np_cpu.prod(input_shape)
        for i in range(self.floor):
            recur_size = get_conf_param(hconfig, 'recur_size')

            ex_inp_dim = timefeats + recur_size
            weight, bias = self.alloc_param_pair([ex_inp_dim, 4*recur_size])
            bias[0*recur_size:1*recur_size] = 1.0

            rnn_info = [inseq, outseq, timesteps1, timefeats, recur_size, use_state]
            list_.append({'w':weight, 'b':bias, 'info':rnn_info})
            
        if outseq:
                output_shape = [timesteps1, recur_size]
            else:
                output_shape = [recur_size]

        return list_, output_shape


    
    def forward_multi_layer_RNN_layer(self, x, hconfig, pm):
        inseq, outseq, timesteps1, timefeats, recur_size, use_state = pm['info']
        mb_size = x.shape[0]

        if inseq:
            x_slices = x[:, 1:, :].transpose([1,0,2])
            lengths = x[:, 0, 0].astype(np.int32)
            max_length = int(np.max(lengths))
        else:
            x_slice = x
            max_length = timesteps1 - 1
            lengths = [max_length] * mb_size
            
        for i in range(self.floor):
            pm[i]{'recurrent':np.zeros([mb_size, recur_size], dtype = 'float32')
            pm[i]{'state' :np.zeros([mb_size, recur_size], dtype = 'float32')}
            pm[i]{'outputs':[]}
            pm[i]{'aux_step':[]}
        aux_steps=[]

        for n in range(max_length):
            if inseq: x_slice = x_slices[n]

            
            for i in range(self.floor):
                pm[i]{'ex_inp':np.hstack([x_slice, pm[i]['recurrent']])}
                pm[i]{'affine':np.matmul(ex_inp, pm[i]['w']) + pm[i]['b']}

                pm[i]{'forget_gate' : sigmoid(affine[:, 0*recur_size:1*recur_size])}
                pm[i]{'input_gate'  : sigmoid(affine[:, 1*recur_size:2*recur_size])}
                pm[i]{'output_gate' : sigmoid(affine[:, 2*recur_size:3*recur_size])}
                pm[i]{'block_input' : tanh   (affine[:, 3*recur_size:4*recur_size])}

                pm[i]{'state_tmp' : state}
                pm[i]{'state': state_tmp * forget_gate + block_input * input_gate}

                pm[i]{'recur_tmp ': tanh(state)}
                pm[i]{'recurrent ': recur_tmp * output_gate}
                x_slice=pm[i]['recurrent']
                if use_state: pm[i]['outputs'].append(pm[i]['state'])
                else: pm[i]['outputs'].append(pm[i]['recurrent']
               
                pm[i]{'aux_step':[pm[i]['ex_inp'], pm[i]['state_tmp'], pm[i]['block_input'], pm[i]['input_gate'], \
                            pm[i]['forget_gate'], pm[i]['output_gate'], pm[i]['recur_tmp']]}
                aux_steps.append(pm[i]['aux_step'])
                if i == len(self.floor)-1:
                    outputs.append(pm[i]['outputs'].append(pm[i]['recurrent'])
                                              
                    

        if outseq:
            output = np.zeros([mb_size, timesteps1, recur_size], dtype = 'float32')
            output[:, 0, 0] = lengths
            output[:, 1:, :] = np.asarray(outputs).transpose([1, 0, 2])
        else:
            output = np.zeros([mb_size, recur_size], dtype = 'float32')
            for n in range(mb_size):
                output[n] = outputs[int(lengths[n]-1)][n]

        return output, [x, lengths, max_length, outputs, aux_steps]


    def backprop_multi_layer_RNN_layer(self, G_y, hconfig, pm, aux):
        inseq, outseq, timesteps1, timefeats, recur_size, use_state = pm['info']
        x, lengths, max_length, outputs, aux_steps = aux
        mb_size = x.shape[0]

        G_weight = np.zeros_like(pm['w'], dtype = 'float32')
        G_bias = np.zeros_like(pm['b'], dtype = 'float32')
        G_x = np.zeros(x.shape, dtype = 'float32')
        G_recurrent = np.zeros([mb_size, recur_size], dtype = 'float32')
        G_state = np.zeros([mb_size, recur_size], dtype = 'float32')

        if inseq: G_x[:, 0, 0] = lengths

        if outseq:
            G_outputs = G_y[:, 1:, :].transpose([1, 0, 2])
        else:
            G_outputs = np.zeros([max_length, mb_size, recur_size], dtype = 'float32')
            for n in range(mb_size):
                G_outputs[lengths[n]-1, n, :] = G_y[n]

        for n in reversed(range(0, max_length)):
            #self.floor 곱연산 해야함.
            if use_state: G_state += G_outputs[n]
            else: G_recurrent += G_outputs[n]

            ex_inp, state_tmp, block_input, input_gate, forget_gate, \
                                  output_gate, recur_tmp = aux_steps[n]

            G_recur_tmp = G_recurrent * output_gate
            G_output_gate = G_recurrent * recur_tmp

            G_state += tanh_derv(recur_tmp) * G_recur_tmp

            G_input_gate = G_state * block_input
            G_block_input = G_state * input_gate

            G_forget_gate = G_state * state_tmp
            G_state = G_state * forget_gate

            G_affine = np.zeros([mb_size, 4*recur_size], dtype = 'float32')


            G_affine[:, 0*recur_size:1*recur_size] = \
                                    sigmoid_derv(forget_gate) * G_forget_gate
            G_affine[:, 1*recur_size:2*recur_size] = \
                                    sigmoid_derv(input_gate)  * G_input_gate
            G_affine[:, 2*recur_size:3*recur_size] = \
                                    sigmoid_derv(output_gate) * G_output_gate
            G_affine[:, 3*recur_size:4*recur_size] = \
                                    tanh_derv   (block_input) * G_block_input

            g_affine_weight = ex_inp.transpose()
            g_affine_input = pm['w'].transpose()

            G_weight += np.matmul(g_affine_weight, G_affine)
            G_bias += np.sum(G_affine, axis=0)
            G_ex_inp = np.matmul(G_affine, g_affine_input)

            if inseq: G_x[:,n+1,:] = G_ex_inp[:, :timefeats]
            else: G_x[:,:] += G_ex_inp[:, :timefeats]

            G_recurrent = G_ex_inp[:, timefeats:]

        self.update_param(pm, 'w', G_weight)
        self.update_param(pm, 'b', G_bias)

        return G_x
